<a href="https://colab.research.google.com/github/julesarar/MLSECArarLegranche/blob/main/MLSEC_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install Jinja2==3.1.2
!pip install -q git+https://github.com/RobustBench/robustbench.git@2d630bc9e8d1cf50d46a4dda65fd36850e3ef769
!pip install torch torchvision
!pip install numpy scipy
import torch
from torch.distributions.normal import Normal

!pip install -q git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack

!pip install -q foolbox  # produces 2 incompatibility messages, but they can be just ignored
import foolbox as fb

In [ ]:
# Define the normal distribution with a mean of 0 and standard deviation of 1
dist = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

def add_noise(x, eps):
    # Define the normal distribution with a mean of 0 and standard deviation of 1
    dist = torch.distributions.normal.Normal(0, 1)
    # Generate random noise from the normal distribution
    noise = dist.rsample(x.shape).to(x.device)
    # Add noise to the image
    x_noisy = x + eps * noise
    # Ensure that the image values are within the range [0, 1]
    x_noisy = torch.clamp(x_noisy, 0, 1)
    return x_noisy

def add_noise_Linf(x, eps):
    # Define the uniform distribution over [-1, 1]
    dist = torch.distributions.uniform.Uniform(-1, 1)
    # Generate random noise from the uniform distribution
    noise = dist.sample(x.shape).to(x.device)
    # Scale the noise by the epsilon parameter
    noise = eps * noise
    # Clip the noisy image to be within [0, 1]
    x_noisy = torch.clamp(x + noise, 0, 1)
    return x_noisy


In [ ]:
##LOAD THE IMAGES
from robustbench.data import load_cifar10
x_test, y_test = load_cifar10(n_examples=50)
eps = 0.00
x_noisy = add_noise(x_test, eps)
x_noisyLinf = add_noise_Linf(x_test, eps)  # Adding random noise


In [ ]:
from robustbench.utils import load_model

#LOAD ROBUSTBENCH MODELS (you cannot load each one of them at the same time considering their size)

#L2 MODELS
'''model1 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', threat_model='L2')'''
'''model2 = load_model(model_name='Rebuffi2021Fixing_R18_cutmix_ddpm', threat_model='L2')'''
'''model3 = load_model(model_name='Ding2020MMA', threat_model='L2')'''

#LINF MODEL
modelLinf = load_model(model_name='Gowal2021Improving_70_16_ddpm_100m', threat_model='Linf')

In [ ]:
'''model_fb1 = fb.PyTorchModel(model1, bounds=(0, 1))
model_fb2 = fb.PyTorchModel(model2, bounds=(0, 1))
model_fb3 = fb.PyTorchModel(model3, bounds=(0, 1))'''
# Create Pytorch models
#UNQUOTE WHICH PyTorch MODEL YOU WANT TO CREATE 
'''model_fb1_noisy = fb.PyTorchModel(model1, bounds=(0, 1))'''
model_fb1Linf_noisy = fb.PyTorchModel(modelLinf, bounds=(0, 1))
'''model_fb2_noisy = fb.PyTorchModel(model2, bounds=(0, 1))
model_fb3_noisy = fb.PyTorchModel(model3, bounds=(0, 1))'''

#UNQUOTE TO EVALUATE THE FIRST L2 MODEL
'''_, advs1noisy, success1noisy = fb.attacks.L2PGD(rel_stepsize=0.1, steps=20)(model_fb1_noisy, x_noisy.to('cuda:0'), y_test.to('cuda:0'), epsilons=[0.5])
#Print the robust accuracy of the model on the adversarial examples
print('Robust accuracy: {:.1%}'.format(1 - success1noisy.float().mean()))
print("\n")'''

#UNQUOTE TO EVALUATE THE SECOND L2 MODEL
'''_, advs2noisy, success2noisy = fb.attacks.L2PGD(rel_stepsize=0.1, steps=20)(model_fb2_noisy, x_noisy.to('cuda:0'), y_noisy.to('cuda:0'), epsilons=[0.5])
print('Robust accuracy: {:.1%}'.format(1 - success2noisy.float().mean()))'''

#UNQUOTE TO EVALUATE THE THIRD L2 MODEL
'''_, advs3noisy, success3noisy = fb.attacks.L2PGD(rel_stepsize=0.1, steps=20)(model_fb3_noisy, x_noisy.to('cuda:0'), y_noisy.to('cuda:0'), epsilons=[0.5])
print('Robust accuracy: {:.1%}'.format(1 - success3noisy.float().mean()))'''

# UNQUOTE TO EVALUATE THE LINF MODEL
'''_, advs1noisy, success1noisy = fb.attacks.L2PGD(rel_stepsize=0.1, steps=20)(model_fb1Linf_noisy, x_noisyLinf.to('cuda:0'), y_test.to('cuda:0'), epsilons=[0.5])
#Print the robust accuracy of the model on the adversarial examples
print('Robust accuracy: {:.1%}'.format(1 - success1noisy.float().mean()))'''

In [ ]:
'''adversary = AutoAttack(model1, norm='L2', eps=0.5, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary.apgd.n_restarts = 1'''

#Generate adversarial examples using the AutoAttack object and evaluate them on the noisy inputs and true labels
'''x_adv = adversary.run_standard_evaluation(x_noisy, y_test)'''


adversary2 = AutoAttack(modelLinf, norm='Linf', eps=0.05, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary2.apgd.n_restarts = 1

#Generate adversarial examples using the AutoAttack object and evaluate them on the noisy inputs and true labels
x_adv2 = adversary2.run_standard_evaluation(x_noisyLinf, y_test)

'''adversary = AutoAttack(model2, norm='L2', eps=0.5, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary.apgd.n_restarts = 1
x_adv2 = adversary.run_standard_evaluation(x_noisy, y_test)

adversary = AutoAttack(model3, norm='L2', eps=0.5, version='custom', attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary.apgd.n_restarts = 1
x_adv2 = adversary.run_standard_evaluation(x_noisy, y_test)'''
